In [8]:
import sys
sys.path.append('./strats')
import pandas as pd
import numpy as np
from backtest_tool import load_plotly_theme
from backtest_tool import plot_strategy_and_benchmark
from backtest_tool import save_strategy_and_benchmark
from strats.strategy import strategy
from strats.rolling_mean_strat import Momentum_Rolling_Mean
from strats.rsi_ibs import RSI_IBS

load_plotly_theme()

In [9]:
bench = pd.read_csv("../01_get_ticker_data/data/BTCUSDT.csv", index_col=0)

In [10]:
bench['2017':'2018']

,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_volume,taker_buy_quote_volume
timestamp,,,,,,,,,,
2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009,2017-08-17 04:59:59.999,2.023661e+05,171,35.160503,1.509525e+05
2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916,2017-08-17 05:59:59.999,1.003048e+05,102,21.448071,9.260828e+04
2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229691,2017-08-17 06:59:59.999,3.128231e+04,36,4.802861,2.079532e+04
2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443249,2017-08-17 07:59:59.999,1.924106e+04,25,2.602292,1.129135e+04
2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807,2017-08-17 08:59:59.999,4.239504e+03,28,0.814655,3.552747e+03
...,...,...,...,...,...,...,...,...,...,...
2017-12-31 19:00:00,13745.00,13875.00,13700.00,13780.16,278.041542,2017-12-31 19:59:59.999000,3.832784e+06,3641,146.704521,2.022868e+06
2017-12-31 20:00:00,13783.09,14050.11,13780.13,13971.95,347.902334,2017-12-31 20:59:59.999000,4.850476e+06,4883,210.649864,2.937955e+06
2017-12-31 21:00:00,13971.95,14005.00,13900.01,13940.99,239.784462,2017-12-31 21:59:59.999000,3.341663e+06,3552,100.188897,1.397096e+06


In [11]:
bench['hourly_return_close'] = bench['close'].diff()
bench['hourly_return_close'] = bench['close'].pct_change(1)
bench['hourly_return_open'] = bench['open'].pct_change(1)


In [12]:
bench

,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_volume,taker_buy_quote_volume,hourly_return_close,hourly_return_open
timestamp,,,,,,,,,,,,
2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009,2017-08-17 04:59:59.999,2.023661e+05,171,35.160503,1.509525e+05,NaN,NaN
2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916,2017-08-17 05:59:59.999,1.003048e+05,102,21.448071,9.260828e+04,0.001506,0.011111
2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229691,2017-08-17 06:59:59.999,3.128231e+04,36,4.802861,2.079532e+04,0.002093,0.004980
2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443249,2017-08-17 07:59:59.999,1.924106e+04,25,2.602292,1.129135e+04,0.005929,-0.003157
2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807,2017-08-17 08:59:59.999,4.239504e+03,28,0.814655,3.552747e+03,0.002460,0.003869
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-17 18:00:00,85108.67,85470.01,84899.71,84899.71,929.239650,2025-04-17 18:59:59.999000,7.923648e+07,120912,396.726970,3.383137e+07,-0.002455,0.002057
2025-04-17 19:00:00,84899.71,85119.47,84495.38,84849.06,870.295450,2025-04-17 19:59:59.999000,7.379743e+07,127392,383.455540,3.251911e+07,-0.000597,-0.002455
2025-04-17 20:00:00,84849.05,85140.00,84849.05,85140.00,383.924320,2025-04-17 20:59:59.999000,3.263660e+07,64881,197.214140,1.676528e+07,0.003429,-0.000597


In [31]:
from backtest_tool import compute_sharpe
#from backtest_tool import backtest_from_strat

trading_fees = 0.001
#strat_arr = [Momentum_Rolling_Mean(), RSI_IBS()]
strat_arr = [Momentum_Rolling_Mean(), RSI_IBS()]
params_strat = {
        'momentum_rolling_mean' : {},
        'RSI' : {'rsi_period' : 40}

}

rah = []
rih= []


def backtest_from_strat(strat_obj, bench, start=None, end=None, params={}, bench_target='hourly_return_open', plot_enable=True, sharp_treshold=0.99, vol_treshold=0.33,
                        return_treshold = 0.01, drawdown_treshold = -0.3, save_plot_enable=True, trading_fees = 0.001):
    """Backtest a strategy from the strategy class

    Args:
        strat_obj (_type_): _description_
        bench (_type_): _description_
        start (_type_, optional): _description_. Defaults to None.
        end (_type_, optional): _description_. Defaults to None.
        params (dict, optional): _description_. Defaults to {}.
        bench_target (str, optional): _description_. Defaults to 'hourly_return_open'.
        plot_enable (bool, optional): _description_. Defaults to True.
        sharp_treshold (float, optional): _description_. Defaults to 0.99.
        vol_treshold (float, optional): _description_. Defaults to 0.33.
        return_treshold (float, optional): _description_. Defaults to 0.01.
        drawdown_treshold (float, optional): _description_. Defaults to -0.3.
        save_plot_enable (bool, optional): _description_. Defaults to True.
        trading_fees (float, optional): _description_. Defaults to 0.001.
    """
    signal = strat_obj.compute_signal(bench, start=start, end=end, **params)
    signal = signal.fillna(0)
    strat = signal * bench[bench_target]
    strat -= (signal.diff().abs() * trading_fees)
    # Metrics calculus
    sharpe = compute_sharpe(strat)
    annualized_returns = (strat.mean() * 365 * 24)
    annualized_vol = (strat.std() * np.sqrt(365 * 24))
    drawdown = strat - strat.cummax()
    max_drawdown = drawdown.min()
    nbr_position = (signal != 0).sum()
    win_position = (strat  > 0).sum()
    loss_position = (strat < 0).sum()
    flat_position = (strat == 0).sum()
    global rah
    global rih
    rah = signal
    rih = strat
    print(f'== \033[96m{strat_obj.name} \033[0m==\n')
    strat_obj.describe()
    print('let\'s print some metrics\n')
    print(f"number of non flat positions: {nbr_position}")
    print(f"number of winning positions: {win_position}")
    print(f"number of loosing positions (including fees after returning flat): {loss_position}")
    print(f"number of flat positions: {flat_position}")
    print(f"Annualized Return: {"\033[92m" if annualized_returns > return_treshold else '\033[91m'} {annualized_returns:.2%}\033[0m==")
    print(f"Annualized Volatility:{"\033[92m" if annualized_vol < vol_treshold else '\033[91m'} {annualized_vol:.2%}\033[0m==")
    print(f"Sharpe Ratio: {"\033[92m" if sharpe > sharp_treshold else '\033[91m'} {sharpe:.2f}\033[0m==")
    print(f"Max Drawdown: {"\033[92m" if max_drawdown > drawdown_treshold else '\033[91m'} {max_drawdown:.2%}\033[0m==\n\n")
    bench_to_plot = bench[bench_target]
    if start and end:
        bench_to_plot = bench_to_plot[start:end]
    elif start:
        bench_to_plot = bench_to_plot[start:]
    elif end:
        bench_to_plot = bench_to_plot[end:]
    if plot_enable:
        plot_strategy_and_benchmark(strat.cumsum(), bench_to_plot.cumsum(), title=strat_obj.name)
    start_str = 'lowest' if start is None else str(start)
    end_str = 'highest' if end is None else str(end)
    if save_plot_enable:
        save_strategy_and_benchmark(strat.cumsum(), bench_to_plot.cumsum(), title=f"{strat_obj.name}",save_path=f'./output/{strat_obj.name}_from_{start_str}_to_{end_str}.png')



for strat in strat_arr:
    backtest_from_strat(strat, bench, params=params_strat[strat.name],plot_enable=False, save_plot_enable=False, trading_fees=trading_fees)


== momentum_rolling_mean ==

Strategy name: momentum_rolling_mean
Asset Targeted: BTCUSDT

The principle of this strategy is simple
We have to windows on long and the other one short
We go long when the mean of the long window applied on the close is lower than the one from the short window
We are short on the opposite
There is no flat on this strategy


let's print some metrics

number of non flat positions: 67090
number of winning positions: 33056
number of loosing positions: 34012
number of flar positions: 22
Annualized Return:  179.98%==
Annualized Volatility: 75.88%==
Sharpe Ratio:  2.37==
Max Drawdown:  -35.59%==


== RSI ==

Strategy name: RSI
Asset Targeted: BTCUSDT

let's print some metrics

number of non flat positions: 1133
number of winning positions: 15
number of loosing positions: 2073
number of flar positions: 65002
Annualized Return:  -175.44%==
Annualized Volatility: 19.58%==
Sharpe Ratio:  -8.96==
Max Drawdown:  -11.75%==




In [71]:
set(rih[(rih < 0)].index) ^ set(rah[(rah != 0)].index)

{'2017-09-28 10:00:00',
 '2017-10-09 22:00:00',
 '2017-10-12 17:00:00',
 '2017-10-12 20:00:00',
 '2017-10-13 01:00:00',
 '2017-10-13 03:00:00',
 '2017-10-13 05:00:00',
 '2017-10-13 11:00:00',
 '2017-10-13 16:00:00',
 '2017-10-30 05:00:00',
 '2017-11-01 11:00:00',
 '2017-11-01 13:00:00',
 '2017-11-02 01:00:00',
 '2017-11-02 06:00:00',
 '2017-11-02 08:00:00',
 '2017-11-02 12:00:00',
 '2017-11-16 07:00:00',
 '2017-11-16 11:00:00',
 '2017-11-16 23:00:00',
 '2017-11-26 06:00:00',
 '2017-11-26 08:00:00',
 '2017-11-26 12:00:00',
 '2017-11-26 14:00:00',
 '2017-11-26 18:00:00',
 '2017-11-26 20:00:00',
 '2017-11-27 02:00:00',
 '2017-11-27 06:00:00',
 '2017-11-27 10:00:00',
 '2017-11-29 06:00:00',
 '2017-11-29 12:00:00',
 '2017-11-29 15:00:00',
 '2017-12-06 07:00:00',
 '2017-12-06 12:00:00',
 '2017-12-07 00:00:00',
 '2017-12-07 01:00:00',
 '2017-12-07 06:00:00',
 '2017-12-07 12:00:00',
 '2017-12-08 03:00:00',
 '2017-12-16 17:00:00',
 '2017-12-16 21:00:00',
 '2017-12-17 00:00:00',
 '2018-01-06 02:

In [57]:
rih[(rih < 0)]

timestamp
2017-09-28 09:00:00   -0.006180
2017-09-28 10:00:00   -0.001000
2017-10-09 21:00:00   -0.009877
2017-10-09 22:00:00   -0.001000
2017-10-12 16:00:00   -0.018374
                         ...   
2025-04-07 00:00:00   -0.001000
2025-04-07 03:00:00   -0.009777
2025-04-07 04:00:00   -0.001000
2025-04-07 07:00:00   -0.028720
2025-04-07 08:00:00   -0.001000
Length: 2073, dtype: float64

In [90]:
rih.loc['2017-09-28 09:00:00':]

timestamp
2017-09-28 09:00:00   -0.00618
2017-09-28 10:00:00   -0.00100
2017-09-28 11:00:00   -0.00000
2017-09-28 12:00:00    0.00000
2017-09-28 13:00:00    0.00000
                        ...   
2025-04-17 18:00:00    0.00000
2025-04-17 19:00:00   -0.00000
2025-04-17 20:00:00   -0.00000
2025-04-17 21:00:00    0.00000
2025-04-17 22:00:00   -0.00000
Length: 66084, dtype: float64

In [87]:
rah.loc['2017-09-28 09:00:00':]

timestamp
2017-09-28 09:00:00   -1.0
2017-09-28 10:00:00    0.0
2017-09-28 11:00:00    0.0
2017-09-28 12:00:00    0.0
2017-09-28 13:00:00    0.0
                      ... 
2025-04-17 18:00:00    0.0
2025-04-17 19:00:00    0.0
2025-04-17 20:00:00    0.0
2025-04-17 21:00:00    0.0
2025-04-17 22:00:00    0.0
Name: signal, Length: 66084, dtype: float64

In [79]:
rah[rah == -1]

timestamp
2017-09-28 09:00:00   -1.0
2017-10-09 21:00:00   -1.0
2017-10-12 16:00:00   -1.0
2017-10-12 19:00:00   -1.0
2017-10-12 23:00:00   -1.0
                      ... 
2025-03-24 11:00:00   -1.0
2025-03-24 14:00:00   -1.0
2025-03-24 16:00:00   -1.0
2025-03-24 18:00:00   -1.0
2025-03-24 19:00:00   -1.0
Name: signal, Length: 741, dtype: float64

In [77]:
rah.loc['2017-09-28 09:00:00']

-1.0

In [64]:
rah.value_counts()

signal
 0.0    65958
-1.0      741
 1.0      392
Name: count, dtype: int64

In [ ]:
t